### NeuralProphet产生背景
大多数时间序列问题需要易于理解的预测。同时，需要有效的预测。这两个愿望导致了一种权衡：可解释性与准确率。准确率的显著提高通常归因于更复杂的模型。然而，复杂性与可解释性存在天然的矛盾。

简而言之，存在两类模型：

* 统计算法（ARIMA、GARCH …）可以在知道理论的情况下进行数学解释
* 神经网络模型，虽然更加复杂，但已经证明了它们的性能

**NeuralProphet = 神经网络 + Prophet**
因此，目的是通过在 Prophet（一种统计算法）中包含神经网络来结合前两个类别。目的：提高准确性，同时限制可解释性的损失

这个介绍部分可以通过下图来大致描述:
<center><img src = './pic/neturalprophet01.png' width=500></center>

**Neural Prophet是升级版的Prophet。NeuralProphet模型集成了Prophet的所有优点，不仅具有不错的可解释性，还有优于Prophet的预测性能。**

* 使用PyTorch作为后端进行优化的梯度下降法。
* 使用AR-Net对时间序列的自相关进行建模。
* 使用seepearate前馈神经网络对滞后回归者进行建模。
* 可配置的FFNNs非线性深层。
* 可调整到特定的预测范围（大于1）。
* 自定义损失和指标。

#### **针对prophet的改变，可总结为以下两大点**

**1 分解的分量增加了一个重量级的AR（自回归），这个是超级重量级的**
* AR模型本身在经典时序分析里面就独当一面，到了Neural Prophet里面做为辅助，它对于精度的提高是神级的。
* 相比于传统的AR模型， AR模型借助于神经网络，拟合的更加快捷和精确。
* 对于AR模型也添加了正则项（参数为ar_sparsity）。通过正则加持， AR模型可以看的更远，因为你有能力使用更多的历史数据而不用担心求解。

**2 Pytorch作为backend**
* Neural Prophet把所有的分量都用pytorch写了
* 趋势项，周期项变化不大，因为他们的算法还是和Prophet一致。
* AR项，外部回归项变化很大，因为有了Pytorch的加持，意味着你不用再局限于线性回归
* 从源码看，Neural Prophet采用Relu非线性化了
* 神经网络适合魔改，看着结果不顺眼，改改层数，说不定就顺眼了。

### NeuralProphet原理
NeuralProphet是一个可分解的时间序列模型，和Prophet相比，类似的组成部分有趋势(trend)、季节性(seasonality)、自回归(auto-regression)、特殊事件(special events)，不同之处在于引入了未来回归项(future regressors)和滞后回归项(lagged regressors)。
* **未来回归项**（future-known regressors）是指在预测期有已知未来值的外部变量，而滞后回归项（lagged covariates）是指那些只有观察期值的外部变量。趋势trend可以通过设置变化点来建立线性或者组合多个线性趋势的模型。季节性seasonality使用傅里叶项建模，因而可以解决高频率数据的多种季节性。
* **自回归项**（auto-regression）使用AR-Net的实现来解决，这是一个用于时间序列的自回归前馈神经网络(Auto-Regressive Feed-Forward Neural Network)。滞后回归项也使用单独的前馈神经网络进行建模。未来回归项和特殊事件都是作为模型的协变量，只要single weight进行建模。



### **NeuralProphet模块组件**
NeuralProphet模型模型由多个模块组成，每个模块都有各自的输入和建模过程。 每个模块都有自己的个人输入和建模过程。 但是，所有模块都必须产生 h 个输出，其中h 定义了一次预测未来的步数。
$$\hat y_t = T(t) + S(t) + E(t) + F(t) + A(t) + L(t)$$
**其中:**
* T(t) = 时间 t 的趋势
* S(t) = 时间 t 的季节性影响
* E(t) = 时间 t 的事件和假日效应
* F(t) = 未来已知外生变量在时间 t 的回归效应
* A(t) = 基于过去观察的时间 t 的自回归效应
* L(t) = t 时刻外生变量滞后观测的回归效应

所有模型组件模块都可以单独配置和组合以组成模型。 如果所有模块都关闭，则仅安装一个静态偏移参数作为趋势分量。 默认情况下，仅激活 trend 和 seasonality 模块
#### **Trent T(t)**
趋势与之前的 Prophet 模型没有变化。简而言之，**<u>我们希望使用指数或线性增长函数来模拟趋势</u>**。使用逻辑增长是一种非常传统且广为接受的解决方案，但是原始 Prophet 模型的创新之处在于它允许函数的参数发生变化。这些变化点由模型动态确定，并为其他静态增长率和偏移参数提供了更大的自由度

建模趋势的经典方法是将其建模为偏移量 m 和增长率 k。 t1时刻的趋势效应由增长率乘以从偏移量 m 顶部的起点 t0 开始的时间差。
$$T(t_1) = T(t_0)Z + k*\Delta_t = m + k*(t_1-t_0)$$

#### **Seasonality S(t)**
季节性被定义为以特定的定期间隔发生的变化。众所周知，它很难解释，因为它可以采用多种形式。该模型的最初开发人员提出了另一个好主意——他们没有尝试使用自回归（即滞后数据）对季节性进行建模，而是尝试对季节性曲线进行建模。这就是傅立叶级数的用武之地。**<u>傅立叶级数只是一堆加在一起的正弦曲线，可用于拟合任何曲线。</u>**一旦我们有了数据的每日、每周、每月等季节性的函数形式，我们就可以简单地将这些术语添加到我们的模型中并准确预测未来的季节性

类似Prophet，NeuralProphet也使用傅立叶项（Fourier terms）来处理Seasonality。傅立叶项定义为正弦、余弦对并允许对多个季节性以及具有非整数周期的季节性进行建模。每个季节性可以定义为多个傅里叶项
$$S_p(t) = \sum_{j=1}^k(a_j*cos(\frac{2\pi jt}p) + b_j*sin(\frac{2\pi jt}p))  $$

其中 k 具有周期性p的季节性的傅立叶的数量。每一个季节性对应 2k 个系数。在 t 时刻，模型涉及的所有季节性效应可以表示为：
$$s(t) = \sum_{p∈P}S_p^*(t)$$

每个季节周期性可以单独表示为 additive 或者 multiplicative 的形式：
$$
S_p^*(t) = 
\begin{cases}
S_p^+(t) = T(t)*S_p(t) & if\ S_p\ is\ multiplicative\\
S_p(t) & otherwise
\end{cases}
$$

NeuralProphet根据数据频率和长度自动激活每日、每周或每年的季节性。 既 周期大于2个季节性，激活对应季节性。默认每个季节性的傅立叶项数为：年：k=6，p=365.25，周： k=3，p=7，日：p=1，k=6。

#### **Events and Holidays E(t)**
对于初始 Prophet 模型中的最后一项，用于处理事件。季节性和事件的处理方式几乎相同 – 使用傅立叶级数。然而，我们希望傅立叶变换不是平滑曲线，而是在给定特定假期时产生非常尖的曲线。而且，由于底层函数是正弦函数，它们很容易扩展到未来。
$$E(t)=\sum_{e∈E}E_e^*(t)$$
其中
$$E_e(t) = z_ee(t)$$
$$
E_p^*(t) = 
\begin{cases}
E_e^+(t) = T(t)*E_e(t) & if\ f\ is\ multiplicative\\
E_e(t) & otherwise
\end{cases}
$$

#### **Auto-Regression**
自回归是回顾先前值并将其用作未来值的预测器的概念。最初的先知模型之所以如此有效，是因为它偏离了这一理念，但是为了利用深度学习，我们必须回归。自回归项使用滞后值来预测未来值
$$y_t = c + \sum_{i=1}^{i=p}\theta_i*y_{t-i} + e_t$$
$$A^t(t),A^t(t+1),...,A^t(t+h-1) = AR-NET(y_{t-1},y_{t-2},...y_{t-p})$$
对NeuralProphet对象的n_lags参数设置一个适当的值，就可以在NeuralProphet中启用AR-Net。

例如，将5个滞后期输入AR-Net，并接收3个步骤作为预测
```python
m = NeuralProphet(
    n_forecasts=3,
    n_lags=5,
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
)
```
* num_hidden_layers参数：增加AR-Net的复杂度
* ar_sparsity参数：为AR-Net加入正则约束
* highlight_nth_step_ahead_of_each_forecast参数：突出特定的预测步骤，用法:
```python
m.highlight_nth_step_ahead_of_each_forecast(step_number=m.n_forecasts)
```

#### **Lagged Regressors L(t)**
Prophet 和 NeuralProphet 模型的强大方面之一是它们允许协变量。大多数时间序列预测模型都是单变量的，尽管它们有时会提供多变量版本——ARIMA 与 MARIMA。
在处理带有时间序列预测的协变量时，我们需要确保这些协变量会提前 n 个时间段出现，否则我们的模型将无法预测。我们可以通过将当前协变量滞后 n 个时间段来做到这一点，这由 L(t) 项建模，或者对这些协变量进行预测，由 F(t) 项建模。

滞后回归用于为我们的时间序列预测目标加入其他观察变量。在 t 时刻，我们只知道t-1时刻（包括t-1）前的观察值。
$$L(t) = \sum_{x\in X}L_x(x_{t-1},x_{t-2},...,x_{t-p})$$
只有在启用AR-Net的情况下，才会支持Lagged Regressor

NeuralProphet通过调用add_lagged_regressor函数并给出必要的设置，将Lagged Regressor用于NeuralProphet对象中。

假如有一个名为A的滞后回归项，通过add_lagged_regressor注册它到NeuralProphet
```python
m = m.add_lagged_regressor(names=’A’)
```
#### **Future Regressors F(t)**
未来回归器是指具有已知未来值的外部变量。从这个意义上说，未来回归者的功能如果和特殊事件非常相似。这些回归器的过去值对应于训练time stamps，必须与训练数据本身一起提供。

对于future regressors，在t时刻，过去和未来的回归因子（regressor） 都是知道的。
$$F(t) = \sum_{f \in F}F_f^*(t)$$
其中：
$$ F_f(t) = d_ff(t)$$
$$
F_f^*(t) = 
\begin{cases}
F_f^+(t) = T(t)*F_f(t) & if\ f\ is\ multiplicative\\
F_f(t) & otherwise
\end{cases}
$$

### **NeuralProphet模型训练**
Prophet 使用Stan 实现的 L-BFGS 拟合模型，NeuralProphet依赖PyTorch实现的SGD拟合模型；

#### **Loss Function 损失函数** 
NeuralProphet默认使用Huber loss，即smooth L1-loss，如下
$$L_{huber}(y,\hat y) = 
\begin{cases}
\frac {1}{2\beta}(y - \hat y)^2 , & for \ |y-\hat y| < \beta \\
|y-\hat y|- \frac {\beta}{2} ,& otherwise
\end{cases}
$$

对于给定的阈值β, 低于阈值β，损失函数变为mean squared error (MSE)；高于阈值β，损失函数变为mean absolute error (MAE)。用户可以设定MSE， MAE 或者其他PyTorch实现的函数；

#### **Regularization 正则化函数**
NeuralProphet使用权重绝对值的缩放和对数变换偏移作为一般正则化函数。

其中$\epsilon \in (0,\infty) $为偏移的倒数，$\alpha \in (0,\infty) $为对数变换的比例。ϵ控制权重参数的稀疏程度，α控制权重值的大小；

默认情况下，设置$ϵ=1,\alpha = 1$

#### **Optimizer 优化器**

**默认使用AdamW**
AdamW 是一种优化算法，它是 Adam 优化算法的一个变种，由 Liu et al. 在 2019 年提出。它结合了 Adam 优化器和权重衰减（weight decay）的概念，旨在提高模型训练的稳定性和性能。以下是 AdamW 函数参数的解释：

* AdamW(params, lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.0001,maximize=False, capturable=False)
	* lr: 学习率（learning rate），用于控制每次迭代更新参数时的步长。默认值为 0.001。
	* betas: 一个元组，包含了两个浮点数，分别代表 Adam 算法中的 β1 和 β2 参数。β1 用于计算一阶矩估计（即梯度的指数加权平均值），β2 用于计算二阶矩估计（即梯度平方的指数加权平均值）。默认值为 (0.9, 0.999)。
	* eps: 一个非常小的正浮点数，用于数值稳定性，防止分母为零。默认值为 1e-8。
	* weight_decay: 权重衰减的强度。这个值与 AdamW 算法中的 β2 相乘，用于控制正则化项的衰减速度。默认值为 0.0001。
	* maximize: 一个布尔值，指示是否最大化目标函数（通常用于强化学习）。默认为 False，表示最小化目标函数。
	* capturable: 一个布尔值，指示优化器是否应该捕获参数的当前状态。默认为 False

**也可以选用SGD**
SGD（随机梯度下降）是一种常用的优化算法，用于机器学习中的参数优化。以下是 SGD 函数参数的解释：
* SGD(params, lr=0.01, momentum=0.9, dampening=0, weight_decay=0.001, nesterov=False)
同时也可以使用PyTorch支持的其他 optimizer
	* lr: 学习率（learning rate），用于控制每次迭代更新参数时的步长。默认值为 0.01。
	* momentum: 动量（momentum）参数，用于加速SGD在正确方向上的收敛速度，并减少震荡。动量项是之前梯度的加权平均。默认值为 0.9。
	* dampening: 阻尼项，用于防止更新步长过大，通常与动量一起使用。阻尼项是动量项的一个变体，可以减少动量的影响。默认值为 0。
	* weight_decay: 权重衰减，用于正则化，可以防止模型过拟合。权重衰减通过在损失函数中添加一个惩罚项来实现，这个惩罚项与参数的平方和成比例。默认值为 0.001。
	* nesterov: Nesterov 动量，一个布尔值，指示是否使用 Nesterov 加速梯度。Nesterov 动量是一种动量方法，它在计算动量时考虑了参数的更新，从而可能提供更快的收敛速度。默认为 False。

SGD 算法通过迭代地调整参数来最小化损失函数。在每次迭代中，SGD 计算损失函数关于参数的梯度，并根据学习率和动量参数更新参数。如果启用了权重衰减，还会在更新中加入一个惩罚项。如果启用了 Nesterov 动量，还会在动量计算中考虑参数的更新。
SGD 是一种简单但有效的优化算法，广泛应用于各种机器学习任务中，尤其是在深度学习中。然而，由于其收敛速度可能较慢，因此在实践中经常使用动量或 Nesterov 动量来加速训练过程

#### **Learning Rate 学习率**
通过测试的方式来估计一个学习率，针对特定的数据集大小，测试100 + log10(10 + T ) ∗ 50) 次, 测试区间从η=1e−7到η=1e+2之间。为了稳定性，分别测试三次求平均：
$$\eta^* = \frac{1}{3}(log_{10}(\eta_1) + log_{10}(\eta_2) + log_{10}(\eta_3)) $$
$$\eta = 10^{\eta^*}$$

#### **Batch Size**
如果未指定，根据数据量T来决定
$$B^* = 2^{2+|log_{10}(T)|}$$
$$B = min(T,max(16,min(256,B^*)))$$

#### **epochs 迭代次数**
如果未指定，根据数据量T来决定
$$N_{epoch}^* = \frac {1000\times 2^{\frac{5}{2}\times log_{10}(T)}}{T}$$

$$N_{epoch} = min(500,max(50,|N_{epoch}^*|))$$

#### **Scheduler**
采用1cycle policy，学习率在最初的30%训练时间内从$\frac{\eta}{100}$上升到η，接着cosine曲线下降到$\frac{\eta}{5000}$，直到训练结束。

参考：https://www.biaodianfu.com/neuralprophet.html

In [ ]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level

# Load the dataset for tutorial 4 with the extra temperature column
# df = pd.read_csv("https://github.com/ourownstory/neuralprophet-data/raw/main/kaggle-energy/datasets/tutorial01.csv")
# df.head()
# df = pd.DataFrame({'ds': pd.date_range(start = '2022-12-01', periods = 10, freq = 'D'),
#                    'y': [9.59, 8.52, 8.18, 8.07, 7.89, 8.09, 7.84, 7.65, 8.71, 8.09]})
#加载数据集
df_value=pd.read_csv('data/bill_2024.csv')
#将列名转换为ds y 。必须是这两个列名，否则报错
df = df_value.rename(columns={'date': 'ds', 'value': 'y'}) 
# 将date类型由字符串转成时间类型 ，必须转
df['ds']=pd.to_datetime(df['ds'])
# df['date']=pd.to_datetime(df['date'])
# Disable logging messages unless there is an error
set_log_level("ERROR")

# Default model
m = NeuralProphet()

m.set_plotting_backend("plotly-static")

# Continue training the model and making a prediction
metrics = m.fit(df)
# 预测数据集
forecast = m.predict(df)
m.plot(forecast)